# 二分类问题——二分类交叉熵损失函数（Binary Cross-Entropy Loss，BCE）

## 数学公式

$$\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(p_i) + (1-y_i)\log(1-p_i) \right]$$  
其中，
  - $y_i \in \{0, 1\}$：样本的真实标签（0或1）。  
  - $p_i \in (0, 1)$：模型预测样本属于类别1的概率（通常由Sigmoid函数输出）。  
  - $N$：样本数量。 

## 核心思想

二分类交叉熵损失函数对每个样本分别计算两个可能结果的加权对数概率：  
  - 当$y_i=1$时，二分类交叉熵损失函数退化为$\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N}\log(p_i)$，相当于仅计算$\log(p_i)$，也即惩罚预测概率$p_i$偏离1的情况。  
  - 当$y_i=0$时，二分类交叉熵损失函数退化为$\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N}\log(1-p_i)$，相当于仅计算$\log(1-p_i)$，也即惩罚预测概率$p_i$偏离0的情况。

## 损失函数特性

1. **概率校准**: 二分类交叉熵损失函数直接优化预测概率与真实标签的匹配程度，而非仅关注分类边界，精度更高。
2. **输出概率值**:  二分类交叉熵损失函数要求模型输出为概率值，而非直接输出类别标签。输出概率值能够更好反映模型预测的可信程度。
3. **训练效率高**: 
   - 预测错误时（如$y_{i}=1$但$p_{i} \to 0$），梯度较大，促使模型快速调整参数。  
   - 预测正确且置信度高时（如$y_{i}=1$且$p_{i} \to 1$），梯度趋近于0，参数更新放缓。  
4. **数值稳定性**:
    - 实际实现时，会通过数值优化（对log函数添加微小值$\epsilon$）避免$\log(0)$导致的数值溢出。

## python实现

**np.clip**: 将数组中的值限定在指定的最小值 (a_min) 和最大值 (a_max) 之间
  - `a`: 输入的数组
  - `a_min`: 数组元素的下界，所有小于 a_min 的值都会被替换为 a_min
  - `a_max`: 数组元素的上界，所有大于 a_max 的值都会被替换为 a_max

In [5]:
import numpy as np

def binary_cross_entropy(y_true, y_pred, epsilon=1e-15):
    """
    手动实现二分类交叉熵损失函数（数值稳定版本）
    
    参数：
    - y_true: 真实标签，形状 (N,)，取值 {0, 1}
    - y_pred: 预测概率，形状 (N,)，取值 (0, 1)
    - epsilon: 极小值，防止 log(0) 溢出
    
    返回：
    - 平均损失值
    """
    # 将预测值限制在 [epsilon, 1-epsilon] 范围内
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    
    # 计算损失
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

y_true = np.array([1.0, 0.0, 1.0])
y_pred = np.array([0.8, 0.2, 0.9])
binary_cross_entropy(y_true, y_pred)

0.18388253942874858

# 多分类问题——多分类交叉熵损失函数（Categorical Cross-Entropy Loss，CCE）

## 数学公式

对于有$C$个类别的分类问题，给定样本的真实标签$y_{i,c}$和模型预测的类别概率$p_{i,c}$，多分类交叉熵损失定义如下：
$$\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \sum_{c=1}^{C} y_{i,c} \log(p_{i,c})$$  
其中：
  - $y_{i,c} \in \{0, 1\}$：样本$i$的真实标签是否为类别$c$，是为1，否为0。（将真实标签转换为 one-hot 编码形式，便于计算）
  - $p_{i,c} \in (0, 1)$：模型预测样本$i$属于类别$c$的概率。  
  - $N$：样本数量，
  - $C$：类别总数。  

## 核心思想

衡量模型预测的概率分布$p$与真实分布$y$之间的差异。
- 如果$y_{i,c} = 1$（即该样本$i$属于类别$c$），则损失函数退化为$-\log(p_{i,c})$，表示模型预测该类别的概率越大，损失越小。
- 如果$y_{i,c} = 0$（即该样本$i$不属于类别$c$），则该类别不会影响损失函数计算。

## 损失函数特性

1. **概率校准**: 多分类交叉熵损失函数直接优化预测概率与真实标签的匹配程度，而非仅关注分类边界，精度更高。
2. **输出概率值**:  多分类交叉熵损失函数要求模型输出为概率值，而非直接输出类别标签。输出概率值能够更好反映模型预测的可信程度。
3. **训练效率高**: 
   - 预测错误时（如$y_{i,c}=1$但$p_{i,c} \to 0$），梯度较大，促使模型快速调整参数。  
   - 预测正确且置信度高时（如$y_{i,c}=1$且$p_{i,c} \to 1$），梯度趋近于0，参数更新放缓。  
4. **数值稳定性**:
    - 实际实现时，会通过数值优化（对log函数添加微小值$\epsilon$）避免$\log(0)$导致的数值溢出。

## python实现

In [19]:
import numpy as np

def categorical_cross_entropy(y_true_labels, y_pred_probs, epsilon=1e-15):
    """
    输入要求：
    - y_true_labels: 真实类别索引（非one-hot），形状 (N,)
    - y_pred_probs: 已归一化的预测概率（每个样本的概率和为1），形状 (N, C)
    - epsilon: 极小值，防止log(0)溢出

    返回：
    - 多分类交叉熵损失
    """

    # 提取真实类别对应的预测概率
    N = y_pred_probs.shape[0] # 获取样本数量
    true_probs = y_pred_probs[np.arange(N), y_true_labels] # 获取每个样本对应类别的预测概率值
    
    # 数值稳定处理：限制概率范围在 [epsilon, 1] 内
    true_probs = np.clip(true_probs, epsilon, 1.0)
    
    # 计算交叉熵
    loss = -np.mean(np.log(true_probs))
    return loss

# 示例数据：3个样本，3个类别
y_true_labels = np.array([0, 2, 1])  # 真实类别索引
y_pred_probs = np.array([
    [0.9, 0.1, 0.0],    # 样本1：正确预测类别0
    [0.2, 0.3, 0.5],    # 样本2：正确预测类别2
    [0.1, 0.7, 0.2]     # 样本3：正确预测类别1
])

# 手动计算损失
manual_loss = categorical_cross_entropy(y_true_labels, y_pred_probs)
print(f"手动实现多分类交叉熵损失: {manual_loss:.4f}")

手动实现多分类交叉熵损失: 0.3851
